<a href="https://colab.research.google.com/github/matt-fidelis-95/BERT-Fake-News/blob/master/artigo_mestrado_portugues_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics import f1_score, accuracy_score , recall_score , precision_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/My Drive/script_python')

In [ ]:

from tqdm import tqdm

from sklearn.model_selection import train_test_split

def fnc(path):

    h = []
    b = []
    l = []
    with open(path,encoding="ISO-8859-1") as fdata:  
        data = fdata.readlines()
    for line in data:
        stripped_line = line.strip()
        li = line.split(';')
        if(len(li) == 3):
          rm_brkl = (li[2]).strip()
          if rm_brkl == '-1':
            l.append(0)
          else:
            l.append(1)
          h.append(li[0])
          b.append(li[1])

    return h, b, l

#headlines, bodies, labels = fnc('Facom_Vitor.csv')
headlines_aux, bodies_aux, labels_aux = fnc('brCorpus.csv')
#headlines.extend(headlines_aux)
#bodies.extend(bodies_aux)
#labels.extend(labels_aux)
list_of_tuples = list(zip(headlines_aux, bodies_aux, labels_aux))#list_of_tuples = list(zip(headlines, bodies, labels))
Corpus = pd.DataFrame(list_of_tuples, columns=['id', 'text', 'label'])


# Passo - a : Remove linhas vazias.
Corpus['text'].dropna(inplace=True)
# Passo - b : Converte todo o texto para minusculo. 
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
# Passo - c : Tokenizacao : Cada entrada do corpus e quebrada em um conjunto de palavras
Corpus['text']= [word_tokenize(entry, language = 'portuguese') for entry in Corpus['text']]
# Passo - d : Remove palavras de parada, nao numericas e faz o Stemming.

for index,entry in enumerate(Corpus['text']):
    Final_words = []
    # Inicializa o RSLPStemmer() que faz o stemming para portugues
    word_RSLPStemmer = RSLPStemmer()
    for word in entry:
        # Checa palavras de parada e considera apenas o alfabeto
        if word not in stopwords.words('portuguese') and word.isalpha():
            word_Final = word_RSLPStemmer.stem(word)
            Final_words.append(word_Final)
    # O conjunto final de palavras para cada iteracao e salvo em 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3, random_state=1234)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(ngram_range=(1,2),max_df= 0.85, min_df= 0.01)#max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
print(Tfidf_vect.vocabulary_)

{'text': 1387, 'promot': 1136, 'ped': 1014, 'lul': 816, 'petist': 1038, 'mp': 894, 'expresid': 572, 'val': 1434, 'forçõ': 632, 'grup': 677, 'pesso': 1036, 'promov': 1137, 'contr': 313, 'luiz': 814, 'silv': 1310, 'mulh': 899, 'maris': 842, 'letçðc': 793, 'luçð': 823, 'filh': 611, 'velh': 1442, 'casal': 219, 'paul': 1008, 'preven': 1107, 'pt': 1157, 'net': 930, 'empreit': 498, 'pinh': 1044, 'oa': 953, 'outr': 981, 'doi': 458, 'investig': 739, 'cas': 217, 'expresid luiz': 574, 'luiz lul': 815, 'lul silv': 819, 'maris letçðc': 843, 'don': 461, 'morr': 887, 'sp': 1332, 'ano': 75, 'intern': 733, 'desd': 397, 'dia': 428, 'jan': 749, 'sofr': 1324, 'acompanh': 19, 'nest': 922, 'sextaf': 1302, 'hospit': 691, 'ter': 1373, 'acid': 16, 'provoc': 1150, 'fot': 636, 'segu': 1272, 'protocol': 1146, 'ofic': 965, 'mort': 888, 'submet': 1336, 'test': 1385, 'prim': 1110, 'ocorr': 960, 'çÿ': 1486, 'segund': 1275, 'determin': 417, 'nest sextaf': 928, 'expresid lul': 575, 'pf': 1041, 'diz': 454, 'fez': 608, '

In [ ]:
print(Train_X_Tfidf)

  (0, 1460)	0.6972839050648637
  (0, 1351)	0.2674025259375641
  (0, 1318)	0.13682883492834833
  (0, 1066)	0.1795255834160773
  (0, 1065)	0.1038872134493406
  (0, 916)	0.13343079598069527
  (0, 899)	0.25748649021861103
  (0, 874)	0.17781135154924593
  (0, 873)	0.1509181682092473
  (0, 657)	0.13151092862811817
  (0, 467)	0.1887729208250111
  (0, 385)	0.12647641309807464
  (0, 313)	0.08839956280500522
  (0, 245)	0.13202379593448432
  (0, 201)	0.32247914403402345
  (0, 43)	0.18786034864745688
  (0, 21)	0.09270916347719332
  (1, 1486)	0.03176956121546657
  (1, 1470)	0.05998849808722012
  (1, 1454)	0.049069584342584405
  (1, 1434)	0.056330090996323835
  (1, 1432)	0.047402488815185266
  (1, 1425)	0.054278058898577786
  (1, 1416)	0.055842952230265336
  (1, 1406)	0.05012211445350184
  :	:
  (5036, 1003)	0.07253365297297552
  (5036, 981)	0.06605984787891547
  (5036, 967)	0.12450729874908215
  (5036, 948)	0.12423593551277134
  (5036, 929)	0.11853399455621234
  (5036, 922)	0.06605984787891547
  (5

In [ ]:
# Classificador - Algoritmo - SVM
# ajusta o conjunto de dados de treinamento no classificador
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# prediz os rotulos no conjunto de dados de validacao
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print ( "F1 score {:.4}%".format( f1_score(Test_Y, predictions_SVM, average='macro')*100 ) )

SVM Accuracy Score ->  89.21296296296296
F1 score 89.21%


In [ ]:
LABELS = [0, 1]
RELATED = [0, 1, 2]

def print_confusion_matrix(cm):
    lines = ['CONFUSION MATRIX:']
    header = "|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)
    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|".format(LABELS[i], *row))
        lines.append("-"*line_len)
    lines.append("ACCURACY: {:.3f}".format((hit / total)*100) + "%")
    print('\n'.join(lines))

def fnc_score_cm(predicted_labels, target):
    score = 0.0
    cm = [[0, 0],
          [0, 0]]
    for i, (g, t) in enumerate(zip(predicted_labels, target)):
            if g == t:
                score += 0.25
                if g != 3:
                    score += 0.50
            if g in RELATED and t in RELATED:
                score += 0.25

            cm[g][t] += 1
    return score,  cm

fnc_score, cm_test = fnc_score_cm(predictions_SVM, Test_Y)
print_confusion_matrix(cm_test)

CONFUSION MATRIX:
-------------------------------------
|           |     0     |     1     |
-------------------------------------
|     0     |    981    |    122    |
-------------------------------------
|     1     |    111    |    946    |
-------------------------------------
ACCURACY: 89.213%
